### Import python packages

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, Button, HBox, Layout, VBox
from IPython.display import display
import ipywidgets as widgets
from chvisual import CHData
from dataset import Dataset
import IPython
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.lines import Line2D

### Create select for the SPARQL endpoint

In [2]:
def set_SPARQL(endpoint):
    print(f'SPARQL is {endpoint}')
    display()
    return endpoint

dataset = Dataset()
     
sparqlWidget = widgets.interactive(set_SPARQL, endpoint=['https://data.bnf.fr/sparql']);

In [3]:
sparqlWidget.result

'https://data.bnf.fr/sparql'

### Init the SPARQL lib and test

In [4]:
chdata = CHData(sparqlWidget.result, 100)

### Retrieve authors from the selected SPARQL endpoint and create the interface

In [5]:
def set_Author(author):

    idAuthor = author[author.rindex('-') +2:]
    authorData = chdata.getAuthorInfo(idAuthor)
    metadataItems = []
    optionsItems = []
 
    p = None
    metadata = ''
    for r in authorData:
        propertyLink = "<a href=" + r['p']['value'] +  "><font color='blue'>"+ r['p']['value'] + "</font></a>"
        metadata = metadata + "<p>" + propertyLink + ": " + r['o']['value'] + "</p>" 
    p = widgets.HTML(metadata)
    metadataItems.append(p)

    img = None
    url = chdata.getDepiction(authorData)
    if url != None and url != '':
        img = widgets.HTML("<img width='50%' src='"+url+"'>")
        optionsItems.append(img)

    downloadButton = widgets.Button(
        description='Download metadata',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Download metadata',
        icon='download' # (FontAwesome names without the `fa-` prefix)
    )

    def on_button_clicked(b):
        chdata.downloadGraph(idAuthor)
        
    downloadButton.on_click(on_button_clicked)
    optionsItems.append(downloadButton)

    optionsLayout = VBox()
    optionsLayout.children = optionsItems
    metadataItems.append(optionsLayout)

    metadataLayout = HBox()
    metadataLayout.children = metadataItems

    widget_plot_graph = widgets.Output(layout=Layout(width='70%'))
    with widget_plot_graph:
        IPython.display.clear_output(True)
        g = chdata.getGraph(idAuthor)

        colors = []
        for u in g.nodes(data="nodetype"):
            if u[1] in ("red", "green", "blue","yellow", "cyan", "magenta"):
                colors.append(u[1])
            elif u[1] == None:
                colors.append("blue")
            else:
                #do something?
                print("ERROR: Should be red, green, blue or None")

        legend_elements = [
            Line2D([0], [0], marker='o', color='w', label='Author class',markerfacecolor='g', markersize=15),
            Line2D([0], [0], marker='o', color='w', label='Work class',markerfacecolor='r', markersize=15),
            Line2D([0], [0], marker='o', color='w', label='Author node',markerfacecolor='y', markersize=15),
            Line2D([0], [0], marker='o', color='w', label='Other',markerfacecolor='b', markersize=15),
            Line2D([0], [0], marker='o', color='w', label='Manifestation class',markerfacecolor='m', markersize=15),
            Line2D([0], [0], marker='o', color='w', label='Concept class',markerfacecolor='c', markersize=15),
        ]
        
        nx.draw_networkx(g, node_color=colors, with_labels=False)
        plt.legend(handles=legend_elements, loc='upper right')
        plt.show()

    worksData = chdata.getAuthorWorks(idAuthor)
    worksWidget = None
    worksText = ''
    for r in worksData:
        workLink = "<a href=" + r['s']['value'] +  "><font color='blue'>"+ r['s']['value'] + "</font></a>"
        worksText = worksText + "<p>" + workLink + ": " + r['label']['value'] + "</p>" 
    worksWidget = widgets.HTML(worksText)

    accordionChildren = []
    accordionChildren.append(metadataLayout)
    accordionChildren.append(worksWidget)
    accordionChildren.append(widget_plot_graph)
    authorLayout = widgets.Accordion(children=accordionChildren, titles=('Metadata', 'Works', 'Graph'))
    display(authorLayout)
    
    return author

authorWidget = widgets.interactive(set_Author, author=chdata.getAuthors());

'name'


### Retrieve classes from the repository

In [6]:
classesItems = []
p = None
classesText = ''
classesData = chdata.getClasses()
for i in classesData[0]:
    classLink = "<a href=" + i +  "><font color='blue'>"+ i + "</font></a>"
    classesText = classesText + "<p>" + classLink + "</p>" 
p = widgets.HTML(classesText)
classesItems.append(p)

widget_plot_classes = widgets.Output(layout=Layout(width='50%'))
with widget_plot_classes:
    IPython.display.clear_output(True)
    X = np.array(list(classesData[1].keys()))
    Y = np.array(list(classesData[1].values()))
    plt.xticks(rotation='vertical')
    fig = plt.bar(X, Y, color ='blue', width = 0.6)
    plt.show()
classesItems.append(widget_plot_classes)

classesLayout = HBox()
classesLayout.children = classesItems

### Retrieve properties from the repository

In [7]:
propertiesItems = []
p = None
propertiesText = ''
propertiesData = chdata.getProperties()

for i in propertiesData[0]:
    propertyLink = "<a href=" + i +  "><font color='blue'>"+ i + "</font></a>"
    propertiesText = propertiesText + "<p>" + propertyLink + "</p>" 
p = widgets.HTML(propertiesText)
propertiesItems.append(p)

widget_plot_properties = widgets.Output(layout=Layout(width='50%'))
with widget_plot_properties:
    IPython.display.clear_output(True)
    X = np.array(list(propertiesData[1].keys()))
    Y = np.array(list(propertiesData[1].values()))
    plt.xticks(rotation='vertical')
    fig = plt.bar(X, Y, color ='blue', width = 0.6)
    plt.show()
propertiesItems.append(widget_plot_properties)

propertiesLayout = HBox()
propertiesLayout.children = propertiesItems

### Create Wikidata visualisation

In [8]:
wikidata_file = open('map/bnf-wikidata-map.html', "r")
wikidataWidget = widgets.HTML(wikidata_file.read())

### Create visual interface

In [9]:
children = []
titles = []
tab = widgets.Tab()
children.insert(0,sparqlWidget)
children.insert(1,authorWidget)
children.insert(2,classesLayout)
children.insert(3,propertiesLayout)
children.insert(4,wikidataWidget)
tab.children = children

titles.insert(0, 'SPARQL endpoint')
titles.insert(1, 'Author')
titles.insert(2, 'Classes')
titles.insert(3, 'Properties')
titles.insert(4, 'Wikidata')
tab.titles = titles

tab